![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI&file=Grounding+Overview+-+Vertex+AI+Search.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Grounding%20Overview%20-%20Vertex%20AI%20Search.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FGrounding%2520Overview%2520-%2520Vertex%2520AI%2520Search.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Grounding%20Overview%20-%20Vertex%20AI%20Search.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Grounding%20Overview%20-%20Vertex%20AI%20Search.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Grounding Overview - Vertex AI Search

This workflow uses [Vertex AI Agent Builder](https://cloud.google.com/generative-ai-app-builder/docs/introduction) To build a search experience with [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction).  

Why Vertex AI Search? Building a search application, or a retrieval augmented generation (RAG) application for context and grounding requires steps like:
- process, annotate, and break content into chunks
- generate embeddings of chunks
- store, index, and retrieve chunks
- rank, rerank retrieval based on query/question/prompt
- generation of answer with grounding
- verifying groundeness and identifying contradiction

Vertex AI Search does all of this, and much more, a service within the solution and provides APIs and client for interaction as use in this workflow.

The alternative of building a search or retrieval application can be done with many other Vertex AI and GCP offerings but there are also easy to use APIs to make this fast and simple.  Read more about these offerings [here](https://cloud.google.com/generative-ai-app-builder/docs/builder-apis#build-rag).

This workflow will walk through:
- Preparing unstructured data for input
    - This workflow shows how to make a metadata file so that any number of files can be store in any number of GCS locations (buckets and folders).
- How to create and work with a data store
    - retrieve/create data stores
    - list documents in data stores
    - immport documents
- How to create a search app
    - retrieve/create a search app
- Get search results
    - results = related documents
    - snippets = brief extract that previews documents relationship to the search
    - extrative answers = short verbatim text extract meant to answer the search
    - extrative segments = longer verbatim text extract meant to answer the search or be used as context for LLMs.
    - summaries = automatic LLM summaries of the search results
- Get Answers
    - control the query and answer phase as well and follow-up questions


>**NOTE:**
>If Vertex AI Agent Builder has not been previously used then part of the setup may need to be completed in the console prior to running this notebook which primarily uses the Python SDK. See [Before you begin](https://cloud.google.com/generative-ai-app-builder/docs/before-you-begin).

todo:
- include answer method examples
- include cleanup section for app and datastore
- screenshots of console views




---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.discoveryengine', 'google-cloud-discoveryengine')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [5]:
!gcloud services enable discoveryengine.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [6]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [7]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [10]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'grounding-overview'

# make this the gcs bucket for storing files
GCS_BUCKET = PROJECT_ID 

# vertex search location
VS_LOCATION = 'global'

Packages

In [27]:
import requests
import base64
import json
import hashlib

from google.cloud import storage

import google.cloud.discoveryengine_v1 as discoveryengine
import google.cloud.discoveryengine_v1alpha as discoveryengine_alpha

Clients

In [61]:
# vertex ai agent builder
API_ENDPOINT = dict(api_endpoint = (f'{VS_LOCATION}-' if VS_LOCATION != 'global' else '') + 'discoveryengine.googleapis.com')
datastore_client = discoveryengine.DataStoreServiceClient(client_options = API_ENDPOINT)
document_client = discoveryengine.DocumentServiceClient(client_options = API_ENDPOINT)
engine_client = discoveryengine.EngineServiceClient(client_options = API_ENDPOINT)
search_client = discoveryengine.SearchServiceClient(client_options = API_ENDPOINT)

# gcs client: assumes bucket already exists
gcs = storage.Client(project = PROJECT_ID)
bucket = gcs.bucket(GCS_BUCKET)

---
## Prompt And Context

The [official rules of baseball](https://img.mlbstatic.com/mlb-images/image/upload/mlb/wqn5ah4c3qtivwx3jatm.pdf), a pdf that is updated annually with the latest changes to the game and published by MLB.


In [12]:
prompt = "what are the dimensions of first base in baseball?"

In [16]:
url = 'https://img.mlbstatic.com/mlb-images/image/upload/mlb/wqn5ah4c3qtivwx3jatm.pdf'
# get the pdf
context_bytes = requests.get(url).content
context_base64 = base64.b64encode(context_bytes).decode('utf-8')

---
## Store Document(s) In GCS

In [106]:
# store pdf in gcs
file_blob = bucket.blob(f"{SERIES}/{EXPERIMENT}/{url.split('/')[-1]}")
file_blob.upload_from_string(context_bytes, content_type = 'application/pdf')

---
## Prepare Document(s) For The Vertex Agent Builder Data Store

There are multiple ways to [prepare data for ingesting](https://cloud.google.com/generative-ai-app-builder/docs/prepare-data) depending on location, volume, how often it will change, and type (website, unstructured, strutred, media, third-party (Slack, ServiceNow, ...), Healthcare FHIR, ....).

Here, the data will be prepared as **Unstructured data in GCS storage**.  Files an be imported as:
- **single file** at GCS URI
- **multiple files** in a GCS 'folder'.  Note that import is nnot recursive so subfolder will not be imported.  For this case or even multiple buckets see the next option:
- **Any number of files** at a single or multiple folders and buckets can be imported with a **metadata** file.  A JSON lines file with one line per file that include the document id and uri as well as optional metadata.


In [107]:
for blob in bucket.list_blobs(prefix = f"{SERIES}/{EXPERIMENT}/{url.split('/')[-1]}"):
    print(blob.name)

applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf


### Document ID's

The name of a file could make a great id.  But what happens if multiple file names exist but in different folders?  To make this managable across files, folders and buckets, adopting a hash of the full file path could be a great practice.  This function is designed to convert the full file path into a hash of length 63.

In [108]:
def generate_id(name):
    hasher =hashlib.sha256()
    hasher.update(name.encode('utf-8'))
    return hasher.hexdigest()[0:63]

In [110]:
generate_id(bucket.name+'/'+blob.name)

'b48b4704ced27a1e9bdee5b1d90479abdb488b2df9fa0b93d554ea250039e18'

### Create Metadata

At a minimum the document `id` and the `content` needs to be provided but optional metadata can also provided as shown here with `structData`:

In [111]:
metadata = []
file_types = ['pdf', 'docx', 'txt', 'html', 'pptx']
content_type = dict(
  pdf = 'application/pdf',
  txt = 'text/plain',
  html = 'text/html',
  docx = 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  pptx = 'application/vnd.openxmlformats-officedocument.presentationml.presentation'
)
for blob in bucket.list_blobs(prefix = f"{SERIES}/{EXPERIMENT}/{url.split('/')[-1]}"):
    folder = blob.name.split('/')[0]
    filename = blob.name.split('/')[-1]
    filetype = blob.name.split('.')[-1].lower()
    filepath = '/'.join(blob.name.split('/')[0:-1])
    if filetype in file_types:
        json_data = dict(
            id = generate_id(blob.name),
            structData = dict(
                title= filename,
                path = filepath,
                location = bucket.name
            ),
            content = dict(
                mimeType = content_type[filetype],
                uri = f'gs://{blob.bucket.name}/{blob.name}'
            )
        )
        metadata.append(json.dumps(json_data))   


In [112]:
metadata

['{"id": "73bee30d37112e8e829720fe2f4a3ff0b450c2a51cd9fe03b5beeea277eed67", "structData": {"title": "wqn5ah4c3qtivwx3jatm.pdf", "path": "applied-genai/grounding-overview", "location": "statmike-mlops-349915"}, "content": {"mimeType": "application/pdf", "uri": "gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf"}}']

### Write Metadata File To GCS

In [113]:
metadata_blob = bucket.blob(f'{SERIES}/{EXPERIMENT}/{SERIES}-{EXPERIMENT}.json')
with metadata_blob.open('w') as f:
    for m in metadata:
        f.write(m + '\n')

In [114]:
f"gs://{bucket.name}/{blob.name}"

'gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf'

---
## Create A Search Data Store

[Creating a search data store](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es) depends on the type of data, in this case an [import from cloud storage](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#cloud-storage) using a metadata file in JSON lines format. 

- [Discoveryengine Python Data Store Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.data_store_service.DataStoreServiceClient)

In [115]:
VS_DATASTORE_ID = f"{SERIES}-{EXPERIMENT}"

### Check For Existing Data Store (and retrieve)

In [116]:
try:
    datastore = datastore_client.get_data_store(
        name = datastore_client.collection_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            collection = 'default_collection'
        ) + f'/dataStores/{VS_DATASTORE_ID}'
    )
    ds_exist = True
except Exception as err:
    ds_exist = False
    
ds_exist

True

### Create New Data Store (if needed)

In [117]:
if not ds_exist:
    ds_create = datastore_client.create_data_store(
        parent = datastore_client.collection_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            collection = 'default_collection'
        ),
        data_store = discoveryengine.DataStore(
            display_name = f"{SERIES}-{EXPERIMENT}",
            industry_vertical = discoveryengine.IndustryVertical.GENERIC,
            solution_types = [discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH],
            content_config = discoveryengine.DataStore.ContentConfig.CONTENT_REQUIRED,
            #document_processing_config = discoveryengine.DocumentProcessingConfig(
            #    #chunking_config = ,
            #    default_parsing_config = discoveryengine.DocumentProcessingConfig.ParsingConfig.DigitalParsingConfig,
            #    parsing_config_overrides = [
            #        {'pdf' :discoveryengine.DocumentProcessingConfig.ParsingConfig.LayoutParsingConfig}
            #    ]
            #)
        ),
        data_store_id = VS_DATASTORE_ID
    )
    response = ds_create.result()
    print(ds_create.operation.name)

### Get The Data Store ID

In [118]:
datastore = datastore_client.get_data_store(
    name = datastore_client.collection_path(
        project = PROJECT_ID,
        location = VS_LOCATION,
        collection = 'default_collection'
    ) + f'/dataStores/{VS_DATASTORE_ID}'
)
datastore

name: "projects/1026793852137/locations/global/collections/default_collection/dataStores/applied-genai-grounding-overview"
display_name: "applied-genai-grounding-overview"
industry_vertical: GENERIC
solution_types: SOLUTION_TYPE_SEARCH
default_schema_id: "default_schema"
content_config: CONTENT_REQUIRED
create_time {
  seconds: 1724189228
  nanos: 173562000
}

In [119]:
datastore.name

'projects/1026793852137/locations/global/collections/default_collection/dataStores/applied-genai-grounding-overview'

In [121]:
datastore_id = datastore_client.parse_data_store_path(datastore.name)
datastore_id

{'project': '1026793852137',
 'location': 'global/collections/default_collection',
 'data_store': 'applied-genai-grounding-overview'}

### List Documents If Prior Data Store

In [125]:
if ds_exist:
    doc_exist = False
    for doc in document_client.list_documents(
        parent = document_client.branch_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            data_store = datastore_id['data_store'],
            branch = 'default_branch'
        )
    ):
        print(doc.content.uri)
        if doc.content.uri == f"gs://{bucket.name}/{file_blob.name}":
            doc_exist = True
            break

doc_exist

gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf


True

### Import Documents (if missing)

- [Discoveryengine Python Document Service Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.document_service.DocumentServiceClient)

In [127]:
if not doc_exist:
    doc_import = document_client.import_documents(
        request = discoveryengine.ImportDocumentsRequest(
            parent = document_client.branch_path(
                project = PROJECT_ID,
                location = VS_LOCATION,
                data_store = datastore_id['data_store'],
                branch = 'default_branch'
            ),
            gcs_source = discoveryengine.GcsSource(
                input_uris = [f"gs://{bucket.name}/{metadata_blob.name}"],
                data_schema = 'document'
            ),
            reconciliation_mode = discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL
        )
    )
    response = doc_import.result()
    operation_metadata = discoveryengine.ImportDocumentsMetadata(doc_import.metadata)
    print(operation_metadata)

### Console View Of Data Store

In [134]:
print(f"Review the data store in the console:\n\nhttps://console.cloud.google.com/gen-app-builder/locations/{VS_LOCATION}/collections/{'default_collection'}/data-stores/{datastore_id['data_store']}/data/documents?project=statmike-mlops-349915")

Review the data store in the console:

https://console.cloud.google.com/gen-app-builder/locations/global/collections/default_collection/data-stores/applied-genai-grounding-overview/data/documents?project=statmike-mlops-349915


---
## Create A Search App

The [Relationship between apps and data stores](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest#app-store-relationship) allows one or more apps to search and retrieve from or more (blended search) data stores.

Here we [creatre an app](https://cloud.google.com/generative-ai-app-builder/docs/create-engine-es) that uses the data store created above.

- [Discoveryengine Python Engine Service Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.engine_service.EngineServiceClient)

In [82]:
VS_APP_ID = f"{SERIES}-{EXPERIMENT}"

### Check For Existing Search App

In [130]:
try:
    app = engine_client.get_engine(
        name = engine_client.engine_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            collection = 'default_collection',
            engine = VS_APP_ID
        )
    )
    app_exist = True
except Exception as err:
    app_exist = False
    
app_exist

True

### Create New Search App (if needed)

In [132]:
if not app_exist:
    app_create = engine_client.create_engine(
        parent = engine_client.collection_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            collection = 'default_collection'
        ),
        engine = discoveryengine.Engine(
            display_name = VS_APP_ID,
            industry_vertical = discoveryengine.IndustryVertical.GENERIC,
            solution_type = discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH,
            search_engine_config = discoveryengine.Engine.SearchEngineConfig(
                search_tier = discoveryengine.SearchTier.SEARCH_TIER_ENTERPRISE,
                search_add_ons = [discoveryengine.SearchAddOn.SEARCH_ADD_ON_LLM],
            ),
            data_store_ids = [datastore_id['data_store']],
        ),
        engine_id = VS_APP_ID
    )
    response = app_create.result()
    operation_metadata = discoveryengine.CreateEngineMetadata(app_create.metadata)
    print(operation_metadata)
    print(app_create.operation.name)

### Console View Of Search App

Opens to the preview tab for easy testing:

In [135]:
print(f"Review the search app in the console:\n\nhttps://console.cloud.google.com/gen-app-builder/locations/{VS_LOCATION}/engines/{datastore_id['data_store']}/preview/search?project=statmike-mlops-349915")

Review the search app in the console:

https://console.cloud.google.com/gen-app-builder/locations/global/engines/applied-genai-grounding-overview/preview/search?project=statmike-mlops-349915


---
## Search: Multiple Methods

The search app can be used to retrieve layers of increasing detailed information, including LLM generated summaries that answer the input question.  This section shows how to use use the SDK to do each type of search.

- [Discoveryengine Python Search Service Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.search_service)

In [139]:
serving_config = search_client.serving_config_path(
    project = PROJECT_ID,
    location = VS_LOCATION,
    data_store = VS_DATASTORE_ID,
    serving_config = 'default_config'
)

---
### Get Search Results 

[Get search results](https://cloud.google.com/generative-ai-app-builder/docs/preview-search-results) using the API.  This approach returns a list of documents that match the search/query, and nothing more.

In [146]:
search_response = search_client.search(
    request = discoveryengine.SearchRequest(
        query = prompt, # pass the user search/question
        page_size = 10, # max documents to return
        serving_config = serving_config,
        # search behavior:
        content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = False
            ),
        ),
        # how queries and spelling are handled:
        query_expansion_spec = discoveryengine.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [155]:
#search_response

Gather results:

In [158]:
search_results = []
for result in search_response.results:
    document = dict(result.document.derived_struct_data) |  dict(result.document.struct_data)
    search_results.append(document)

In [160]:
#search_results

Format and Print results:

In [172]:
for r, result in enumerate(search_results):
    print(f"\nSource Document {r+1}:", end = "")
    print(f"\n\tName: {result['title']}", end = "")
    print(f"\n\tLink: {result['link']}", end = "")


Source Document 1:
	Name: wqn5ah4c3qtivwx3jatm.pdf
	Link: gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf

---
### Get Search Results - Including Snippets

In addition to search results you can also request **snippets**, brief extracts from the matched documents that serve as a preview of the content. Read more here: [Get Snippets And Extractive Segments](https://cloud.google.com/generative-ai-app-builder/docs/snippets).

In [178]:
search_response = search_client.search(
    request = discoveryengine.SearchRequest(
        query = prompt, # pass the user search/question
        page_size = 10, # max documents to return
        serving_config = serving_config,
        # search behavior:
        content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = True
            ),
        ),
        # how queries and spelling are handled:
        query_expansion_spec = discoveryengine.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [179]:
#search_response

Gather results:

In [180]:
search_results = []
for result in search_response.results:
    document = dict(result.document.derived_struct_data) |  dict(result.document.struct_data)
    if 'snippets' in document.keys():
        document['snippets'] = [dict(snippet) for snippet in document['snippets'] if snippet['snippet_status'] == 'SUCCESS']
    search_results.append(document)

In [181]:
#search_results

Format and Print results:

In [182]:
for r, result in enumerate(search_results):
    print(f"\nSource Document {r+1}:", end = "")
    print(f"\n\tName: {result['title']}", end = "")
    print(f"\n\tLink: {result['link']}", end = "")
    if 'snippets' in result.keys():
        for s, snippet in enumerate(result['snippets']):
            print(f"\n\tSnippet {s+1}: {snippet['snippet']}", end = "")


Source Document 1:
	Name: wqn5ah4c3qtivwx3jatm.pdf
	Link: gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf
	Snippet 1: All <b>measurements</b> from home <b>base</b> shall be taken from the point where the <b>first</b> and third <b>base</b> lines intersect. The catcher&#39;s box, the <b>batters</b>&#39; boxes, the coaches&nbsp;...

---
### Get Search Results - Including Extrative Answers

In addition to search results you can also request **Extractive Answers**, a short verbatim text extracted from the document for use as a brief answer. Read more here: [Get Snippets And Extractive Segments](https://cloud.google.com/generative-ai-app-builder/docs/snippets).

In [194]:
search_response = search_client.search(
    request = discoveryengine.SearchRequest(
        query = prompt, # pass the user search/question
        page_size = 10, # max documents to return
        serving_config = serving_config,
        # search behavior:
        content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = False
            ),
            extractive_content_spec = discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
                max_extractive_answer_count = 2
            )
        ),
        # how queries and spelling are handled:
        query_expansion_spec = discoveryengine.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [195]:
#search_response

Gather results:

In [196]:
search_results = []
for result in search_response.results:
    document = dict(result.document.derived_struct_data) |  dict(result.document.struct_data)
    if 'extractive_answers' in document.keys():
        document['extrative_answers'] = [dict(answer) for answer in document['extractive_answers']]
    search_results.append(document)

In [197]:
#search_results

Format and Print results:

In [198]:
for r, result in enumerate(search_results):
    print(f"\nSource Document {r+1}:", end = "")
    print(f"\n\tName: {result['title']}", end = "")
    print(f"\n\tLink: {result['link']}", end = "")
    if 'extractive_answers' in result.keys():
        for a, answer in enumerate(result['extrative_answers']):
            print(f"\n\tAnswer {a+1} (page = {answer['pageNumber']}): {answer['content']}", end = "")


Source Document 1:
	Name: wqn5ah4c3qtivwx3jatm.pdf
	Link: gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf
	Answer 1 (page = 14): See Appendix 1. When location of home base is determined, with a steel tape measure 127 feet, 33 ⁄8 inches in desired direction to establish second base. From home base, measure 90 feet toward first base; from second base, measure 90 feet toward first base; the intersection of these lines establishes first base.
	Answer 2 (page = 15): 2.02 Home Base Home base shall be marked by a five-sided slab of whitened rubber. It shall be a <b>17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8½ inches and the remaining two sides are 12 inches</b> and set at an angle to make a point.

---
### Get Search Results - Including Extrative Segments

In addition to search results you can also request **Extractive Segments**, a longer verbatim text (than extrative answers) extracted from the document for use as a brief answer or post-processing like context for an LLM. Read more here: [Get Snippets And Extractive Segments](https://cloud.google.com/generative-ai-app-builder/docs/snippets).

In [200]:
search_response = search_client.search(
    request = discoveryengine.SearchRequest(
        query = prompt, # pass the user search/question
        page_size = 10, # max documents to return
        serving_config = serving_config,
        # search behavior:
        content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = False
            ),
            extractive_content_spec = discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
                max_extractive_segment_count = 2,
                return_extractive_segment_score = True,
                #num_previous_segments = 1,
                #num_next_segments = 1
            )
        ),
        # how queries and spelling are handled:
        query_expansion_spec = discoveryengine.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [201]:
#search_response

Gather results:

In [204]:
search_results = []
for result in search_response.results:
    document = dict(result.document.derived_struct_data) |  dict(result.document.struct_data)
    if 'extractive_segments' in document.keys():
        document['extractive_segments'] = [dict(segment) for segment in document['extractive_segments']]
    search_results.append(document)

In [209]:
#search_results

Format and Print results:

In [212]:
for r, result in enumerate(search_results):
    print(f"\nSource Document {r+1}:", end = "")
    print(f"\n\tName: {result['title']}", end = "")
    print(f"\n\tLink: {result['link']}", end = "")
    if 'extractive_segments' in result.keys():
        for s, segment in enumerate(result['extractive_segments']):
            len_content = len(segment['content'])
            content = segment['content'][0:min(50, len_content)]
            if len_content > 50:
                content += f' ... ({len_content - 50} more characters)'
            content = content.replace("\n", " ")
            print(f"\n\tSegment {s+1} (page = {segment['pageNumber']}, relevance score = {segment['relevanceScore']:.3f}): {content}", end = "")


Source Document 1:
	Name: wqn5ah4c3qtivwx3jatm.pdf
	Link: gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf
	Segment 1 (page = 14, relevance score = 0.831): 2  Rule 2.01  2.00–THE PLAYING FIELD  2.01 Layout  ... (2120 more characters)
	Segment 2 (page = 15, relevance score = 0.814): 3  Rule 2.01 to 2.02  The foul lines and all other ... (1994 more characters)

---
### Get Search Results - Including Summaries

Going a step farther than search results is a search summary.  This request that parts of the results be interpreted by an LLM and summarized as an answer.

- [Get Search Summaries](https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries)
- [Available Model Versions and Lifecycle](https://cloud.google.com/generative-ai-app-builder/docs/answer-generation-models)

In [217]:
search_response = search_client.search(
    request = discoveryengine.SearchRequest(
        query = prompt, # pass the user search/question
        page_size = 10, # max documents to return
        serving_config = serving_config,
        # search behavior:
        content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = False
            ),
            summary_spec = discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count = 5, # how many of page_size to use
                include_citations = True,
                ignore_adversarial_query = True,
                ignore_non_summary_seeking_query = False,
                model_spec = discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                    version = 'stable' # see link above for all options, including specific models/versions
                )
            ),            
        ),
        # how queries and spelling are handled:
        query_expansion_spec = discoveryengine.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [219]:
#search_response

In [221]:
search_response.summary.summary_text

"First base in baseball is located 90 feet from home plate and 90 feet from second base. The intersection of these lines establishes first base. Home base is marked by a five-sided slab of whitened rubber. The slab is a 17-inch square with two corners removed, leaving one edge 17 inches long, two adjacent sides 8.5 inches, and the remaining two sides 12 inches. The distance between the pitcher's plate and home base is 60 feet, 6 inches. \n"

---
### Get Search Results - Including combinations of snipppets, answers, segments, and summaries

While the section above introduced the individual search results components, this section shows that mutliple types or even all types can be combined in a single request.

In [222]:
search_response = search_client.search(
    request = discoveryengine.SearchRequest(
        query = prompt, # pass the user search/question
        page_size = 10, # max documents to return
        serving_config = serving_config,
        # search behavior:
        content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec = discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet = True
            ),
            extractive_content_spec = discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
                max_extractive_answer_count = 2,
                max_extractive_segment_count = 2,
                return_extractive_segment_score = True,
                #num_previous_segments = 1,
                #num_next_segments = 1
            ),
            summary_spec = discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count = 5, # how many of page_size to use
                include_citations = True,
                ignore_adversarial_query = True,
                ignore_non_summary_seeking_query = False,
                model_spec = discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                    version = 'stable' # see link above for all options, including specific models/versions
                )
            ), 
        ),
        # how queries and spelling are handled:
        query_expansion_spec = discoveryengine.SearchRequest.QueryExpansionSpec(
            condition = discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO
        ),
        spell_correction_spec = discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode = discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        )
    )
)

In [223]:
#search_response

Gather results:

In [224]:
search_results = []
for result in search_response.results:
    document = dict(result.document.derived_struct_data) |  dict(result.document.struct_data)
    if 'snippets' in document.keys():
        document['snippets'] = [dict(snippet) for snippet in document['snippets'] if snippet['snippet_status'] == 'SUCCESS']
    if 'extractive_answers' in document.keys():
        document['extrative_answers'] = [dict(answer) for answer in document['extractive_answers']]
    if 'extractive_segments' in document.keys():
        document['extractive_segments'] = [dict(segment) for segment in document['extractive_segments']]
    search_results.append(document)

In [225]:
#search_results

Format and Print results:

In [226]:
for r, result in enumerate(search_results):
    print(f"\nSource Document {r+1}:", end = "")
    print(f"\n\tName: {result['title']}", end = "")
    print(f"\n\tLink: {result['link']}", end = "")
    if 'snippets' in result.keys():
        for s, snippet in enumerate(result['snippets']):
            print(f"\n\tSnippet {s+1}: {snippet['snippet']}", end = "")
    if 'extractive_answers' in result.keys():
        for a, answer in enumerate(result['extrative_answers']):
            print(f"\n\tAnswer {a+1} (page = {answer['pageNumber']}): {answer['content']}", end = "")
    if 'extractive_segments' in result.keys():
        for s, segment in enumerate(result['extractive_segments']):
            len_content = len(segment['content'])
            content = segment['content'][0:min(50, len_content)]
            if len_content > 50:
                content += f' ... ({len_content - 50} more characters)'
            content = content.replace("\n", " ")
            print(f"\n\tSegment {s+1} (page = {segment['pageNumber']}, relevance score = {segment['relevanceScore']:.3f}): {content}", end = "")
            
print(f"\n\nSummary:\n\t{search_response.summary.summary_text}")


Source Document 1:
	Name: wqn5ah4c3qtivwx3jatm.pdf
	Link: gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf
	Snippet 1: All <b>measurements</b> from home <b>base</b> shall be taken from the point where the <b>first</b> and third <b>base</b> lines intersect. The catcher&#39;s box, the <b>batters</b>&#39; boxes, the coaches&nbsp;...
	Answer 1 (page = 14): See Appendix 1. When location of home base is determined, with a steel tape measure 127 feet, 33 ⁄8 inches in desired direction to establish second base. From home base, measure 90 feet toward first base; from second base, measure 90 feet toward first base; the intersection of these lines establishes first base.
	Answer 2 (page = 15): 2.02 Home Base Home base shall be marked by a five-sided slab of whitened rubber. It shall be a <b>17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8½ inches and the remaining two sides are 12 inches</b> and set at

---
## Answers: Get Answers With Follow-Ups

Another search method is `answer` and this section covers the `answer method` using the [Answer API](https://cloud.google.com/generative-ai-app-builder/docs/reference/rest/v1/projects.locations.dataStores.servingConfigs/answer) directly via REST.

The answer method gives control over the query phase, the answer phase, and ability to configure follow-up questions.  For a breakdown of the methods and information on which scenarios to use, or not use, a method check out [Get answers and follow-ups](https://cloud.google.com/generative-ai-app-builder/docs/answer).

In [ ]:
#search_response

Gather results:

Format and Print results: